# Senators notebook

Parses datasets/senado/senators_with_memberships-**termid**.json creating / updating the following polare entites:
* Agents
* Memberships (Senate, Parties)
* Organizations (Senate)
* Posts (Senate)
* Roles (Afiliate, Senator)

## 1. Imports and Constants Declaration

In [17]:
import json   # reads scrapped data
import pandas as pd # helps with file management and visualization
from uuid import uuid4 # creates new uuids for Posts

termid = 55 # legislatura

PROV = 'sen'
FILE_SUFFIX = termid
SUB_DIR = '{:}/'.format(termid)
JSON_PATH =  'datasets/senado/'

AGENTS_PATH = 'datasets/slp/agents/'
MEMBERSHIPS_PATH = 'datasets/slp/memberships/'
ORGANIZATIONS_PATH = 'datasets/slp/organizations/'
ROLES_PATH = 'datasets/slp/roles/'
POSTS_PATH = 'datasets/slp/posts/'


SENADO_URI = '81311052-e5b6-46fe-87ba-83865fa0ffb0' 


 ## 2. Getting the Parties

In [2]:
filename = 'parties.csv'
file_path = '{:}{:}'.format(ORGANIZATIONS_PATH, filename)
df_parties = pd.read_csv(file_path, sep= ';', encoding= 'utf-8', index_col=None)
df_parties.set_index('sigla', inplace=True)


parties_d = df_parties['slp:resource_uri'].to_dict()                        
print({k:parties_d[k]
    for k in ['PT', 'PMDB', 'DEM']})

{'PT': '7b2138a6-df61-11e7-baf1-c82a144c0a85', 'PMDB': '7b210cfa-df61-11e7-9864-c82a144c0a85', 'DEM': '7b20ed7e-df61-11e7-bb65-c82a144c0a85'}


## 3. Reads scrapped json

In [18]:
filename = 'senator_with_memberships-{:}.json'.format(FILE_SUFFIX) 
file_path = '{:}{:}'.format(JSON_PATH, filename)
with open(file_path, mode='r') as f:
    senatorsstr = f.read()
f.close()

senators_with_memberships = json.loads(senatorsstr)

## 4. Agents

In [19]:
mapping = {'skos:prefLabel': 'sen:CodigoParlamentar',
  'foaf:name':'sen:NomeCompletoParlamentar',
  'rdfs:label':'sen:NomeParlamentar',
  'agent_resource_uri':'slp:resource_uri'           
}

agents = [] 
for swm_d in senators_with_memberships:
    agent_d = {mapping[k]: swm_d.get(k, None) for k in mapping}
    if agent_d:
        agents.append(agent_d)

filename = 'senators-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(AGENTS_PATH, SUB_DIR, filename)
df = pd.DataFrame.from_dict(agents)    
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)
df.head()

,sen:CodigoParlamentar,sen:NomeCompletoParlamentar,sen:NomeParlamentar,slp:resource_uri
0,5573,Abel Rebouças São José,Abel Rebouças,828a0e78-b457-47de-8112-a7cce8d24563
1,739,Ciro Nogueira Lima Filho,Ciro Nogueira,59888f36-89fc-40e9-adf7-e95bb4906eb2
2,5108,José Aparecido dos Santos,Cidinho Santos,94be5f0e-2cc9-488b-bb32-335a44eb4f1b
3,5136,Cesar Antonio de Souza,Cesar Antonio de Souza,40260ab9-9261-46b9-8ab3-3c569dfa8b24
4,5623,Christopher Belchior Goulart,Christopher Goulart,ad36f1e7-f24a-4f51-9f44-c45468e0d4a1


 ## 5. Senate Org
 
 ### 5.1 Membership Senator
 #### 5.1.1 Process

In [20]:
mapping = {
    'skos:prefLabel': 'sen:CodigoMandato',
    'natureza': 'sen:UfParlamentar',
    'legislatura': 'NumeroLegislatura', 
    'startDate': 'sen:startDate', 
    'finishDate': 'sen:finishDate',
    'membership_resource_uri':'slp:resource_uri'
}

memberships = [] 
for swm_d in senators_with_memberships:
    member_uri = swm_d['agent_resource_uri'] 
    terms = swm_d['terms']
    if terms:
        for term_d in terms:
            memberships_d = {mapping[k]: term_d.get(k, None) for k in mapping}
            if memberships_d:
                memberships_d['org:member'] = member_uri
                memberships_d['org:role'] = str(uuid4())
                memberships.append(memberships_d)



df = pd.DataFrame.from_dict(memberships)    
df.head()            

,NumeroLegislatura,org:member,org:role,sen:CodigoMandato,sen:UfParlamentar,sen:finishDate,sen:startDate,slp:resource_uri
0,55,828a0e78-b457-47de-8112-a7cce8d24563,2e32f585-ce98-432c-b32d-2425cf502072,492,BA,2019-01-31,2015-02-01,61e28c9c-c93b-40fc-a73d-3a963632df24
1,56,828a0e78-b457-47de-8112-a7cce8d24563,35b95149-c35d-4d45-8057-105003c26b32,492,BA,2023-01-31,2019-02-01,669716c0-cf09-4d77-896b-8497c3943b34
2,54,59888f36-89fc-40e9-adf7-e95bb4906eb2,3b643a26-92cd-4762-94dd-642cb5e4113a,458,PI,2015-01-31,2011-02-01,869d897d-3ab6-448a-ba58-8c18a2aa4421
3,55,59888f36-89fc-40e9-adf7-e95bb4906eb2,cdba7ead-8aab-4044-8017-6c80016b02c7,458,PI,2019-01-31,2015-02-01,155310d6-40c8-4c04-9234-6f312193b877
4,54,94be5f0e-2cc9-488b-bb32-335a44eb4f1b,c1ac8055-56f0-49b9-987b-26d91d7d92e1,456,MT,2015-01-31,2011-02-01,3e9c9056-8f75-4d44-b5a3-da45f12caa33


 #### 5.1.2 Save

In [21]:
filename = 'memberships_with_senate-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(MEMBERSHIPS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

  ### 5.2 Roles at Senado
  #### 5.2.1 Process

In [22]:
df['org:role'].values
df = df['org:role'].to_frame()
df = df.rename({'org:role':'slp:resource_uri'}, axis='columns')
df['rdfs:label'] = 'Senador'
df.head()

,slp:resource_uri,rdfs:label
0,2e32f585-ce98-432c-b32d-2425cf502072,Senador
1,35b95149-c35d-4d45-8057-105003c26b32,Senador
2,3b643a26-92cd-4762-94dd-642cb5e4113a,Senador
3,cdba7ead-8aab-4044-8017-6c80016b02c7,Senador
4,c1ac8055-56f0-49b9-987b-26d91d7d92e1,Senador


 ####  5.2.2 Save

In [23]:
filename = 'senator_roles-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(ROLES_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

### 5.3 Posts

#### 5.3.1 Posts Process

In [24]:
df = df.rename({'slp:resource_uri':'org:role'}, axis='columns')
df['slp:resource_uri'] = [str(uuid4()) for _ in range(df.shape[0])]
df['org:postIn'] = [SENADO_URI]*df.shape[0]
df.drop('rdfs:label', axis=1, inplace=True)
df.head()

,org:role,slp:resource_uri,org:postIn
0,2e32f585-ce98-432c-b32d-2425cf502072,4ebfe4d5-64c8-47c3-a75b-7514ec31b93f,81311052-e5b6-46fe-87ba-83865fa0ffb0
1,35b95149-c35d-4d45-8057-105003c26b32,886b789d-12a0-4c08-8942-842fa1c98d54,81311052-e5b6-46fe-87ba-83865fa0ffb0
2,3b643a26-92cd-4762-94dd-642cb5e4113a,3b6189f8-f973-4995-bf5b-8c5aa3f8da07,81311052-e5b6-46fe-87ba-83865fa0ffb0
3,cdba7ead-8aab-4044-8017-6c80016b02c7,bb4087dc-920f-435d-8dfb-18284a39e016,81311052-e5b6-46fe-87ba-83865fa0ffb0
4,c1ac8055-56f0-49b9-987b-26d91d7d92e1,56fb18c2-9067-4ba1-8768-6ebf6cb124ee,81311052-e5b6-46fe-87ba-83865fa0ffb0


#### 5.3.3 Posts save

In [10]:
filename = 'senator_posts-{:}.csv'.format(PROV, FILE_SUFFIX) 
file_path = '{:}{:}{:}'.format(POSTS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

 ## 6. Parties Org

 ### 6.1 Membership
 
 #### 6.1.1 Process

In [11]:
mapping = {
    'sigla': 'sen:SiglaPartido',
    'startDate': 'sen:DataFiliacao' ,
    'finishDate': 'sen:DataDesfiliacao',
    'resource_uri': 'slp:resource_uri'
}

memberships = [] 
for swm_d in senators_with_memberships:
    affiliations = swm_d['affiliations']
    if affiliations:
        for affiliation_d in affiliations:
            if affiliation_d['sigla'] in parties_d:
                memberships_d = {mapping[k]: affiliation_d.get(k, None) for k in mapping}
                if memberships_d:
                    memberships_d['org:member'] = member_uri
                    memberships_d['org:role'] = str(uuid4())
                    memberships.append(memberships_d)



                    
df = pd.DataFrame.from_dict(memberships)    
roles = df['org:role'].values
parties = df['sen:SiglaPartido'].values
df.head()            

,org:member,org:role,sen:DataDesfiliacao,sen:DataFiliacao,sen:SiglaPartido,slp:resource_uri
0,85e8b745-786b-4782-a8a1-e3b3164a3f22,d071b5be-0d81-4d91-a766-990c1599993f,None,2015-02-01,PDT,0577ae6b-9fa4-4ff0-ac19-9dd5cd3598d1
1,85e8b745-786b-4782-a8a1-e3b3164a3f22,c39a8c03-4119-41b9-b904-0a76a45a3c14,None,2004-02-14,PP,b2ec603a-26b8-4e59-8a59-624d2f0973ac
2,85e8b745-786b-4782-a8a1-e3b3164a3f22,01e8b2ab-0196-4b4c-9127-579e6962f6ce,2004-02-13,1999-02-01,PFL,3cf1400f-9fe7-4552-99ac-d0a4a52a1510
3,85e8b745-786b-4782-a8a1-e3b3164a3f22,e2756b98-c268-46f0-a731-5cbe03f1aeb4,None,2011-02-03,PR,da9f5ea7-2636-4240-af94-94e0df597ae9
4,85e8b745-786b-4782-a8a1-e3b3164a3f22,4b5f4cfb-c5f8-4571-90a8-541447aa1a62,None,2011-02-03,DEM,89a85f62-3dce-499c-a72a-0917f5bf5240


 #### 6.1.2 Save

In [12]:
filename = 'memberships_with_parties-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(MEMBERSHIPS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

  ### 6.2 Roles at Parties
  #### 6.2.1 Process

In [13]:
df = df['org:role'].to_frame()
df = df.rename({'org:role':'slp:resource_uri'}, axis='columns')
df['rdfs:label'] = 'Afiliado'
df.head()

,slp:resource_uri,rdfs:label
0,d071b5be-0d81-4d91-a766-990c1599993f,Afiliado
1,c39a8c03-4119-41b9-b904-0a76a45a3c14,Afiliado
2,01e8b2ab-0196-4b4c-9127-579e6962f6ce,Afiliado
3,e2756b98-c268-46f0-a731-5cbe03f1aeb4,Afiliado
4,4b5f4cfb-c5f8-4571-90a8-541447aa1a62,Afiliado


 #### 6.2.2 Save

In [14]:
filename = 'roles_with_affiliation-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(ROLES_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)

 ### 6.3 Posts at Parties
 #### 6.3.1 Process

In [15]:
df = df.rename({'slp:resource_uri':'org:role'}, axis='columns')
df['slp:resource_uri'] = [parties_d.get(sigla, None) for sigla in parties]
df.drop('rdfs:label', axis=1, inplace=True)
df.head()

,org:role,slp:resource_uri
0,d071b5be-0d81-4d91-a766-990c1599993f,7b210070-df61-11e7-97e9-c82a144c0a85
1,c39a8c03-4119-41b9-b904-0a76a45a3c14,7b21152e-df61-11e7-bdcf-c82a144c0a85
2,01e8b2ab-0196-4b4c-9127-579e6962f6ce,7b21046c-df61-11e7-8094-c82a144c0a85
3,e2756b98-c268-46f0-a731-5cbe03f1aeb4,7b211a38-df61-11e7-b1b8-c82a144c0a85
4,4b5f4cfb-c5f8-4571-90a8-541447aa1a62,7b20ed7e-df61-11e7-bb65-c82a144c0a85


 #### 6.3.2 Save

In [16]:
filename = 'posts_with_parties-{:}.csv'.format(PROV) 
file_path = '{:}{:}{:}'.format(POSTS_PATH, SUB_DIR, filename)
df.to_csv(file_path, sep=';', encoding='utf-8', index=None)